In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
! pwd

/content


# ***Set parameters***

In [ ]:
#Name of the Clustering algorithm
__algo__ = "KMEANS"               

#Name of the Word Embeddings used (glove, w2v, ftt), MUST set directory below
__emb__ = "ftt" #@param ["glove", "w2v", "ftt", "use"] {allow-input: true}

#Name of Sentence Embedding algorithm used
__sentemb__ = "pmeans5" #@param ["pmeans5", "normalmean", "use"] {allow-input: true}           

#Number of records to be read from files
recnum =                      2#@param {type: "number"} 

# # #Number of Clusters
k = 350

# Number of records
records = 30000

# Size of sentence embedding
if __emb__ == "w2v":
  features = 500
if __emb__ == "glove":
  features = 250
if __emb__ == "ftt":
  features = 1500
if __emb__ == "use":
    features == "512"

if __sentemb__ == "normalmean":
  features = int(features/5)

#list of megadfs
megadfs = []

#list of corresponding models
models = []

# usesqrt = False                   #@param ["False", "True"] {type:"raw", allow-input: true}

# #Random Sampling to be True/False for records which are read
randomsample = True              #@param ["False", "True"] {type:"raw", allow-input: true} 

#Directory where embeddings are saved for that selected embedding
embedDir = "./drive/My Drive/ColabOutput/FTXSentEmbs" #@param ["./drive/My Drive/ColabOutput/FTXSentEmbs", "./drive/My Drive/ColabOutput/W2VSentEmbs", "./drive/My Drive/ColabOutput/MegaSentEmbs"] {allow-input: true}


#Directory where models are saved
modelDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/models/"           

#Directory Where Megadf is to be saved
megadfDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/MegaDfs/"         

#Directory where plots are saved
plotDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/plots/"           

#Directory where performance and distribution params are to be stored
metadataDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/modelMetaData/" 

#Directory where test outcomes are saved
dumpDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/dump/"              

#Model name
name = "{}_{}_{}_K{}_R{}_F{}".format(__algo__, __emb__, __sentemb__, k, records, features)
modelname = "{}_model.pkl".format(name)

#Model Address
modeladd = modelDir+modelname
print(modeladd)

# ClusterDF name
clusterdfname = "{}_clustered_megadf.pkl".format(name)

# Clusterdf address
clusterdfadd = megadfDir+clusterdfname
print(clusterdfadd)

# Directory where result csv's are stored
resultsDir = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/results/"              

# Result CSV name
resultname = "{}_T{}_results.csv".format(name, recnum)

#Result file address
resultfileadd = resultsDir+resultname

print(resultfileadd)


/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/models/KMEANS_ftt_pmeans5_K350_R30000_F1500_model.pkl
/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/MegaDfs/KMEANS_ftt_pmeans5_K350_R30000_F1500_clustered_megadf.pkl
/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/results/KMEANS_ftt_pmeans5_K350_R30000_F1500_T2_results.csv


# Actual Code

### imports and time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import cluster, datasets
from sklearn.metrics import silhouette_score, davies_bouldin_score
import seaborn as sns
import os, subprocess, sys
import datetime, time
import pickle
from scipy.spatial.distance import cosine 


## Load the megadfs

In [ ]:
tempmdf = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/MegaDfs/Abstract_Abstract_KMEANS_use_use_K350_R30000_F512_clustered_megadf.pkl")


In [ ]:
tempmdf.head()

,id,Title,Abstract,Topic,Title_Embedding,Abstract_Embedding,clusterlabel_title,clusterlabel_abs
26299,http://arxiv.org/abs/1508.00761v1,Recognition of Emotions using Kinects,Psychological studies indicate that emotional ...,psychology,"[0.053678866, -0.009632988, -0.07267857, 0.005...","[0.020883536, -0.031098202, -0.066623844, 0.01...",20,252
8393,http://arxiv.org/abs/quant-ph/0604072v1,Brief History of Quantum Cryptography: A Perso...,Quantum cryptography is the only approach to p...,cryptography,"[0.020271683, -0.036928527, -0.049796335, -0.0...","[-0.046857614, -0.043386728, -0.048527252, -0....",102,178
15272,http://arxiv.org/abs/cond-mat/0409378v2,Geographical threshold graphs with small-world...,Many real networks are equipped with short dia...,geography,"[0.011794036, -0.05427748, -0.012652266, -0.00...","[-0.05856386, -0.04959274, -0.064852424, -0.00...",164,139
11476,http://arxiv.org/abs/0906.2611v2,SEAMONSTER: A Demonstration Sensor Web Operati...,A sensor web is a collection of heterogeneous ...,education,"[-0.0042912737, 0.04556092, -0.057276912, -0.0...","[-0.026967276, 0.05848821, -0.03232752, -0.059...",169,326
19085,http://arxiv.org/abs/0806.4827v3,2-D color code quantum computation,We describe in detail how to perform universal...,logic,"[0.047185894, -0.06582154, -0.053225275, -0.00...","[-0.059069965, -0.034812808, -0.010245204, -0....",338,280


In [ ]:
sampledf = tempmdf.sample(2, random_state=69)
sampledf

,id,Title,Abstract,Topic,Title_Embedding,Abstract_Embedding,clusterlabel_title,clusterlabel_abs
28944,http://arxiv.org/abs/1611.01962v1,High-Resolution Semantic Labeling with Convolu...,Convolutional neural networks (CNNs) have rece...,semantics,"[0.054926984, -0.03251063, -0.031229218, -0.05...","[0.014008306, -0.05429825, -0.03890696, -0.062...",93,11
518,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,archaeology,"[0.013196699, 0.03508813, 0.013127662, -0.0555...","[-0.047361284, 0.040542364, 0.0077921017, -0.0...",269,331


In [ ]:
model ="/content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_model_new.pkl"   #@param {type: "string"}


megadf = "/content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_clustered_megadf.pkl"   #@param {type: "string"}

#store names

model_name = model
megadf_name = megadf

#Load Megadf
print("Loading megadf {}".format(megadf_name))
megadf = pickle.load(open(megadf_name, "rb"))
print("{} loading done!".format(megadf_name))
 
#Load model
print("Loading model {}".format(model_name))
model = pickle.load(open(model_name, "rb"))
print("{} loading done!".format(model_name))







Loading megadf /content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_clustered_megadf.pkl
/content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_clustered_megadf.pkl loading done!
Loading model /content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_model_new.pkl
/content/drive/My Drive/ColabOutput/arxiv_glove/Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_model_new.pkl loading done!


In [ ]:
megadf.head()

,id,Title,Abstract,Topic,Title_Embedding,Abstract_Embedding,clusterlabel_title,clusterlabel_abs
21837,http://arxiv.org/abs/1803.07682v1,A Feature-Driven Active Framework for Ultrasou...,A reliable Ultrasound (US)-to-US registration ...,neurology,"[0.4475177777777778, 0.05546344444444446, 0.12...","[0.23881284531516372, 0.0025751437923080575, 0...",53,4
2718,http://arxiv.org/abs/0704.3853v1,Fluorescence microscopy of single autofluoresc...,In this paper we review the applicability of a...,biology,"[0.2681670559083188, 0.12890394363986055, 0.38...","[0.3078997253086889, 0.11462310730583875, 0.11...",1,134
7341,http://arxiv.org/abs/hep-th/0403052v2,Entropy Bounds and Dark Energy,Entropy bounds render quantum corrections to t...,cosmology,"[0.18931425, 0.4385825, 0.11791142500000001, 0...","[0.24607449733595432, 0.24399425671709019, 0.2...",220,342
15716,http://arxiv.org/abs/0710.4494v1,Mean-value property on manifolds with minimal ...,"Let (M,g) be a non-compact and complete Rieman...",harmonics,"[0.34303456522637193, 0.3023412675798373, 0.17...","[0.3208850447266551, 0.22531258348331698, 0.07...",285,169
9090,http://arxiv.org/abs/1312.6321v2,The virus of my virus is my friend: ecological...,Virophages are viruses that rely on the replic...,ecology,"[0.648300975857385, -0.1613049765602169, -0.01...","[0.42954541646774863, -0.023605147673582048, 0...",135,117


In [ ]:
megadf.columns

Index(['id', 'Title', 'Abstract', 'Topic', 'Title_Embedding',
       'Abstract_Embedding', 'clusterlabel_title', 'clusterlabel_abs'],
      dtype='object')

In [ ]:
megadf.iloc[28944]

id                                     http://arxiv.org/abs/1404.3785v1
Title                 Reducing the Barrier to Entry of Complex Robot...
Abstract              Developing robot agnostic software frameworks ...
Topic                                                          robotics
Title_Embedding       [0.5503432657064702, 0.009375289497669328, 0.1...
Abstract_Embedding    [0.2876241740996425, -0.14520031104398326, 0.2...
clusterlabel_title                                                  333
clusterlabel_abs                                                    245
Name: 26846, dtype: object

In [ ]:
l = list(sampledf["id"])
l

['http://arxiv.org/abs/1611.01962v1', 'http://arxiv.org/abs/1309.6589v3']

In [ ]:
megadf["id"].iloc[0] in l

False

In [ ]:
sampledf = megadf[megadf["id"].isin(l) ]
sampledf = sampledf.iloc[:-1, :]
sampledf


,id,Title,Abstract,Topic,Title_Embedding,Abstract_Embedding,clusterlabel_title,clusterlabel_abs
28498,http://arxiv.org/abs/1611.01962v1,High-Resolution Semantic Labeling with Convolu...,Convolutional neural networks (CNNs) have rece...,semantics,"[-0.04270571428571427, -0.08312285714285712, 0...","[0.11632750997868145, -0.007022299927966316, 0...",29,8
72,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,archaeology,"[0.3902666666666667, 0.6073000000000001, -0.60...","[0.24949665317062777, -0.007534590037428, -0.1...",180,11


In [ ]:
predata = sampledf["Abstract_Embedding"]
data = np.matrix(predata.to_list())
print(data.shape)

(2, 250)


# Start Predicting

## Predict labels for the input abstracts

In [ ]:
pred_labels = model.predict(data, )
pred_labels

array([ 8, 11], dtype=int32)

### Store labels in the df

In [ ]:
sampledf["Abstract_pred_labels"] = pred_labels
sampledf

,id,Title,Abstract,Topic,Title_Embedding,Abstract_Embedding,clusterlabel_title,clusterlabel_abs,Abstract_pred_labels
28498,http://arxiv.org/abs/1611.01962v1,High-Resolution Semantic Labeling with Convolu...,Convolutional neural networks (CNNs) have rece...,semantics,"[-0.04270571428571427, -0.08312285714285712, 0...","[0.11632750997868145, -0.007022299927966316, 0...",29,8,8
72,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,archaeology,"[0.3902666666666667, 0.6073000000000001, -0.60...","[0.24949665317062777, -0.007534590037428, -0.1...",180,11,11


## Create Result Dictionary

In [ ]:
result_dict = {
    "id": [],
    "Title": [],
    "Abstract": [],
    "Rank_1_id": [],
    "Rank_1_Title": [],
    "Rank_1_Abstract": [],
    "Rank_1_Topic":[],
    "Rank_1_cosdis": [],
    "Rank_2_id": [],
    "Rank_2_Title": [],
    "Rank_2_Abstract": [],
    "Rank_2_Topic":[],
    "Rank_2_cosdis": [],
    "Rank_3_id": [],
    "Rank_3_Title": [],
    "Rank_3_Abstract": [],
    "Rank_3_Topic":[],
    "Rank_3_cosdis": [],
    "Rank_4_id": [],
    "Rank_4_Title": [],
    "Rank_4_Abstract": [],
    "Rank_4_Topic":[],
    "Rank_4_cosdis": [],
    "Rank_5_id": [],
    "Rank_5_Title": [],
    "Rank_5_Abstract": [],
    "Rank_5_Topic":[],
    "Rank_5_cosdis": [],
}

## Cosine Distance function



In [ ]:
def cos_dis(x, y):    
    return cosine(x, y)

## Filling Result dictionary

In [ ]:
for i in range(int(sampledf.shape[0])):
  newabs_clustter_df = pd.DataFrame()
#   paper_data = db.getpapers([int(megadf["id"].iloc[i]),])[0]
  result_dict["id"].append(sampledf["id"].iloc[i])
  result_dict["Title"].append(sampledf["Title"].iloc[i])
  result_dict["Abstract"].append(sampledf["Abstract"].iloc[i])
  print(result_dict)
  print("@@@")
  newabs_label = int(sampledf["Abstract_pred_labels"].iloc[i])
  newabs_clustter_df = megadf[megadf["clusterlabel_abs"] == newabs_label]
  print("###")
  newabs_clustter_df["Dist2New"] = newabs_clustter_df["Abstract_Embedding"].apply(cos_dis, args = (sampledf["Abstract_Embedding"].iloc[i],))
  print("$$$$")
  newabs_clustter_df = newabs_clustter_df.sort_values(by = "Dist2New")
  
  related_papers = newabs_clustter_df.head()
  print("%%%")
  for j in range(1, related_papers.shape[0]+1):
    result_dict["Rank_{}_id".format(j)].append(related_papers["id"].iloc[j-1])
    result_dict["Rank_{}_Title".format(j)].append(related_papers["Title"].iloc[j-1])
    result_dict["Rank_{}_Abstract".format(j)].append(related_papers["Abstract"].iloc[j-1])
    result_dict["Rank_{}_Topic".format(j)].append(related_papers["Topic"].iloc[j-1])
    result_dict["Rank_{}_cosdis".format(j)].append(related_papers["Dist2New"].iloc[j-1])
    print("^^^^")


{'id': ['http://arxiv.org/abs/1611.01962v1'], 'Title': ['High-Resolution Semantic Labeling with Convolutional Neural Networks'], 'Abstract': ['Convolutional neural networks (CNNs) have received increasing attention overthe last few years. They were initially conceived for image categorization,i.e., the problem of assigning a semantic label to an entire input image.  In this paper we address the problem of dense semantic labeling, whichconsists in assigning a semantic label to every pixel in an image. Since thisrequires a high spatial accuracy to determine where labels are assigned,categorization CNNs, intended to be highly robust to local deformations, arenot directly applicable.  By adapting categorization networks, many semantic labeling CNNs have beenrecently proposed. Our first contribution is an in-depth analysis of thesearchitectures. We establish the desired properties of an ideal semanticlabeling CNN, and assess how those methods stand with regard to theseproperties. We observe

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


## Convert Result Dictionary to Dataframe
### Save the dataframe as csv in Results dir

In [ ]:
print(result_dict)


{'id': ['http://arxiv.org/abs/1611.01962v1', 'http://arxiv.org/abs/1309.6589v3'], 'Title': ['High-Resolution Semantic Labeling with Convolutional Neural Networks', 'Mind: An Archaeological Perspective'], 'Abstract': ['Convolutional neural networks (CNNs) have received increasing attention overthe last few years. They were initially conceived for image categorization,i.e., the problem of assigning a semantic label to an entire input image.  In this paper we address the problem of dense semantic labeling, whichconsists in assigning a semantic label to every pixel in an image. Since thisrequires a high spatial accuracy to determine where labels are assigned,categorization CNNs, intended to be highly robust to local deformations, arenot directly applicable.  By adapting categorization networks, many semantic labeling CNNs have beenrecently proposed. Our first contribution is an in-depth analysis of thesearchitectures. We establish the desired properties of an ideal semanticlabeling CNN, an

In [ ]:
resultsdf = pd.DataFrame(result_dict)
resultsdf.head()

,id,Title,Abstract,Rank_1_id,Rank_1_Title,Rank_1_Abstract,Rank_1_Topic,Rank_1_cosdis,Rank_2_id,Rank_2_Title,Rank_2_Abstract,Rank_2_Topic,Rank_2_cosdis,Rank_3_id,Rank_3_Title,Rank_3_Abstract,Rank_3_Topic,Rank_3_cosdis,Rank_4_id,Rank_4_Title,Rank_4_Abstract,Rank_4_Topic,Rank_4_cosdis,Rank_5_id,Rank_5_Title,Rank_5_Abstract,Rank_5_Topic,Rank_5_cosdis
0,http://arxiv.org/abs/1611.01962v1,High-Resolution Semantic Labeling with Convolu...,Convolutional neural networks (CNNs) have rece...,http://arxiv.org/abs/1611.01962v1,High-Resolution Semantic Labeling with Convolu...,Convolutional neural networks (CNNs) have rece...,semantics,0.0,http://arxiv.org/abs/1610.01944v3,PetroSurf3D - A Dataset for high-resolution 3D...,The development of powerful 3D scanning hardwa...,archaeology,0.015738,http://arxiv.org/abs/1906.01290v1,Relational Reasoning using Prior Knowledge for...,Exploiting relationships among objects has ach...,semantics,0.017205,http://arxiv.org/abs/1803.06067v1,Dynamic-structured Semantic Propagation Network,Semantic concept hierarchy is still under-expl...,semantics,0.018573,http://arxiv.org/abs/1912.13282v1,Medusa: A C++ Library for solving PDEs using S...,"Medusa, a novel library for implementation of ...",philosophy,0.019061
1,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,anthropology,0.0,http://arxiv.org/abs/1309.6589v3,Mind: An Archaeological Perspective,What can relics of the past tell us about the ...,archaeology,0.000000,http://arxiv.org/abs/1912.00914v2,"A Changing Dichotomy: The Conception of the ""M...",This short essay traces the conceptual history...,history,0.025871,http://arxiv.org/abs/0906.0328v1,Rivisiting Token/Bucket Algorithms in New Appl...,We consider a somehow peculiar Token/Bucket pr...,computer+science,0.028157,http://arxiv.org/abs/1511.08033v1,On Mathematical Symbols in China,When studying the history of mathematical symb...,mathematics,0.029859


In [ ]:
resultfileadd = "/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/results/new/"+ model_name.split("/")[-1][:-4] + "_results" + ".csv"
resultfileadd

'/content/drive/My Drive/Colab Notebooks/BTechProject_Venkatesh/results/new/Abstract_KMEANS_glove_pmeans5_K350_R30000_F250_model_new_results.csv'

In [ ]:
resultsdf.to_csv(resultfileadd)

In [1]:
! pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.1.5          
blis                          0.4.

In [2]:
import sklearn


In [3]:
sklearn.__version__

'0.22.2.post1'